In [ ]:
#Melakukan import library yang dibutuhkan
import os
import tensorflow as tf
import requests
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalMaxPool2D, concatenate
import matplotlib.pyplot as plt
import shutil
import random
import cv2
from PIL import Image
import numpy as np
from keras.layers import BatchNormalization
from keras import Model
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Sequential


In [ ]:
#import syntax augmentor dan opendataset
!pip install Augmentor --quiet
import Augmentor
!pip install tqdm
from tqdm import tqdm
!pip install opendatasets
import opendatasets as od

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
od.download("https://www.kaggle.com/datasets/jonathanrjpereira/rice-disease")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: muhammadyusufridho
Your Kaggle Key: ··········


100%|██████████| 4.55G/4.55G [02:30<00:00, 32.5MB/s]


##Data Preparation


In [ ]:
#membuat label source path, train dan valid
source_path = 'rice-disease/Rice_All'
#membuat label source train
source_train = os.path.join(source_path, 'train')
#membuat label source valid
source_test = os.path.join(source_path, 'valid')

In [ ]:
#membuat training path dan testing path sekaligus membuat folder nya sekaligus
training_path = 'rice-disease/train'
testing_path = 'rice-disease/test'

#membuat folder baru bernama data

try:
  os.mkdir(testing_path)
except:
  print('path exist')

try:
  os.mkdir(training_path)
except:
  print('path exist')

list_class = os.listdir(training_path)
print(list_class)

[]


In [ ]:
#kita melihat kategori apa saja yang ada didalam data tersebut
categories = os.listdir(source_path)
categories

['Healthy', 'LeafBlast', 'BrownSpot', 'Hispa']

In [ ]:
# membuat folder di data training path maupun testing path sesuai dengan list kategori yang sudah dibuat
for category in categories:
  try:
    os.mkdir(os.path.join(training_path, category))
    os.mkdir(os.path.join(testing_path, category))
  except:
    print('path exist!')

In [ ]:
#menentukan label kelas dari masing masing kategori yang sudah di urutkan
#SET LABEL CLASS
dic_labels = {}
categories.sort()
for index, category in enumerate(categories):
  dic_labels[category] = str(index)
dic_labels

{'BrownSpot': '0', 'Healthy': '1', 'Hispa': '2', 'LeafBlast': '3'}

In [ ]:
# melihat lagi jumlah isinya ?
dic_class = {'class' : [], 'total_img' : []}
for category in categories :
  total_img = os.listdir(os.path.join(testing_path, category))
  dic_class['class'].append(category)
  dic_class['total_img'].append(len(total_img))
dic_class


{'class': ['BrownSpot', 'Healthy', 'Hispa', 'LeafBlast'],
 'total_img': [523, 523, 523, 523]}

In [ ]:
#mindahin gambar folder train
ROOT_DIR = os.path.abspath(os.curdir)
os.chdir(ROOT_DIR)
for i in range(len(dic_class['class'])):
  os.chdir(ROOT_DIR)
  folder_path = os.path.join(source_path, dic_class['class'][i] )
  list_img = os.listdir(folder_path)
  for img in list_img:
    if 'JPG' in img or 'jpg' in img :
      file_path = os.path.join(folder_path, img)
      image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
      resized = cv2.resize(image, (224, 224), interpolation = cv2.INTER_AREA)
      os.chdir(os.path.join(training_path, dic_class['class'][i]))
      cv2.imwrite(img, resized)
      os.chdir(ROOT_DIR)
os.chdir(ROOT_DIR)

In [ ]:
#mindahin gambar ke folder testing
ROOT_DIR = os.path.abspath(os.curdir)
os.chdir(ROOT_DIR)
for i in range(len(dic_class['class'])):
  os.chdir(ROOT_DIR)
  folder_path = os.path.join(source_path, dic_class['class'][i] )
  list_img = os.listdir(folder_path)
  for img in list_img:
    if 'JPG' in img or 'jpg' in img :
      file_path = os.path.join(folder_path, img)
      image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
      resized = cv2.resize(image, (224, 224), interpolation = cv2.INTER_AREA)
      os.chdir(os.path.join(testing_path, dic_class['class'][i]))
      cv2.imwrite(img, resized)
      os.chdir(ROOT_DIR)
os.chdir(ROOT_DIR)

In [ ]:
#augmentation
def perbanyak_(ini, sebanyak_ini):
  source_dir = ini
  output_dir = "."
  p = Augmentor.Pipeline(source_directory=source_dir, output_directory=output_dir)
  p.rotate(probability=1, max_left_rotation=5, max_right_rotation=5)
  p.skew_left_right(probability=0.7, magnitude=0.1)

  p.sample(sebanyak_ini)


jumlah_data = 800
for category in os.listdir(os.path.join(training_path)):
  cof = os.path.join(training_path,category)
  numb = len(os.listdir(cof))
  
  perbanyak_(cof, jumlah_data - numb)

In [ ]:
#membagi x test dan y test
X_test = []
y_test = []

list_category = os.listdir(testing_path)
list_category.sort()
for category in list_category :
  list_image = os.listdir(os.path.join(testing_path, category))
  for image in list_image :
    img = cv2.imread(os.path.join(testing_path, category, image), cv2.IMREAD_UNCHANGED)
    resized = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
    img = np.array(resized)
    X_test.append(img)
    y_test.append(dic_labels[category])

In [ ]:
X_test = np.array(X_test)
X_test.shape

(2092, 224, 224, 3)

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(training_path,  image_size=(224, 224))

Found 3200 files belonging to 4 classes.


In [ ]:
#splitting Data
train_size = int(len(data)*.7)
val_size = len(data)-train_size

In [ ]:
#membuat label train dan val
train = data.take(train_size)
val = data.skip(train_size).take(val_size)

##Model VGG19

In [ ]:
base_model = VGG19(include_top = False, weights='imagenet', input_shape=(224, 224, 3))

80134624/80134624 [==============================] - 2s 0us/step


In [ ]:
#memangkas 5 bagian bawah dari vgg19 sesuai jurnal
base_model = tf.keras.models.Sequential(base_model.layers[:-5])

In [ ]:
#membuat inception module
def inception_module(layer_in, f1, f2, f3):
	# 1x1 conv
	conv1 = Conv2D(f1, (1,1), padding='same', activation='swish')(layer_in)
	# 3x3 conv
	conv3 = Conv2D(f2, (3,3), padding='same', activation='swish')(layer_in)
	# 5x5 conv
	conv5 = Conv2D(f3, (5,5), padding='same', activation='swish')(layer_in)
	# 3x3 max pooling
	pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
	# concatenate filters, assumes filters/channels last
	layer_out = concatenate([conv1, conv3, conv5], axis=-1)
	return layer_out
  

In [ ]:
output_new_layer = base_model.output
base_model.output.shape


TensorShape([None, 14, 14, 512])

In [ ]:
base_model.input.shape

TensorShape([None, 224, 224, 3])

In [ ]:
#menambahkan layer baru ke model vgg19
output_new_layer = BatchNormalization(input_shape=(7,7, 512))(output_new_layer)
output_new_layer = inception_module(output_new_layer, 64, 64, 128)
#output_new_layer = inception_module(output_new_layer, 64, 64, 128)
output_new_layer = GlobalMaxPool2D()(output_new_layer)
predictions = Dense(len(list_class), activation='softmax', name='pred') (output_new_layer)
new_model = Model(inputs=base_model.input, outputs = predictions)


In [ ]:
#melakukan frezee sd bloc4_pool
for layer in new_model.layers:
  layer.trainable = False
  if 'block4_pool' in layer.name:
    break

In [ ]:
#persiapan melakukan compile dengan optimizer, loss dan metrics
new_model.compile(optimizer = 'adam', loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, verbose=1, restore_best_weights=True)

In [ ]:
#melakukan training dengan label hist
hist = new_model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])